# Домашнее задание №2: Сравнение тэггеров

Делаем соревнование тэггеров: с помощью трёх библиотек размечаем корпус слов и сравниваем с собственноручно размеченным Золотым Стандартом.

Для этого нам понадобятся следующие библиотеки:

In [2]:
import csv
import time
import re
from string import punctuation
import conllu
from nltk import word_tokenize
import pandas as pd
import stanza
from ufal.udpipe import Model, Pipeline  # pylint: disable=no-name-in-module
from pymorphy2 import MorphAnalyzer

## Разметка текста

Прочтём файл с текстом, который мы взяли из интернета. Токенизируем его с помощью nltk и запишем в удобный формат для разметки:

In [28]:
with open ('text.txt', encoding='utf-8') as text:
    text = text.read()

text = text.replace('…', '')

list_tokens = word_tokenize(text)

print(list_tokens[:10])
print('Всего токенов:', len(list_tokens))

['Первое', 'что', 'хочется', 'сказать', ',', 'так', 'это', 'то', ',', 'что']
Всего токенов: 365


In [ ]:
list_csv_rows = [['word', 'POS-tag']]

for token in list_tokens:
    list_row = [token, '']
    list_csv_rows.append(list_row)

In [ ]:
with open('corpora.csv', "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    for line in list_csv_rows:
        writer.writerow(line)

1. nltk, в принципе, токенизирует текст достаточно хорошо, сохраняя нужную пунктуацию и не разрывая слова в ненужных местах.
2. Убирать пунктуацию нет смысла, так как все используемые нами библиотеки (о них далее) умеют разбираться с пунктуацией. Делать это разве что можно для того, чтобы сократить усилия на разметку текста.

*Файлы с текстом (``text.txt``) и размеченным корпусом (``corpora.csv``) можно найти в репозитории*

## Обзор библиотек

В данном домашнем задании мы будем сравнивать три библиотеки: ``udpipe``, ``stanza`` и ``pymorphy2``

### ``udpipe``

``udpipe`` - это библиотека от Института Формальной и Прикладной Лингвистики (Карлов Университет, Чешская Республика). Библиотека, используя модели для языков, токенизирует, лемматизирует, ставит морфологические теги, а также строит деревья зависимостей слов в предложениях. Используется CoNLL-U формат для выдачи результатов. Видимо, не смотрит на контекст(?) Морфологические теги - это теги разметки Universal Dependencies.

### ``stanza``

Библиотека от Stanford NLP Group (команда разработчиков и исследователей компьютерной лингвистики в Университете Стэнфорда, США). Используется нейронный пайплайн, основанный на модели языка. Так же может токенизировать (multi-word tokenization), лемматизировать, ставить морфологические теги и теги зависимостей. ``stanza`` также имеет инстурменты для выделения именованных сущностей. POS-теги основаны на разметке UD.

### ``pymorphy2``

Морфологический анализатор для русского и украинского языка от Михаила Коробова. С помощью словаря OpenCorpora умеет слова и отдавать все возможные их разборы, для русского языка выдавая так же вероятность правильности такого разбора. Отдаёт также нормальную форму, умеет склонять и согласовывать по числу. Не смотрит на контекст. Формат POS-тегов такой же, как в OpenCorpora.

### О токенизации

Для ``pymorphy2`` можно использовать токенизатор ``nltk``. ``udpipe`` имеет собственный токенизатор, но раз он так же не смотрит на контекст, токенизатор можно использовать и другой. ``stanza`` предполагает использование собственного токенизатора для разметки по морфологическим тегам.

В связи с вышесказанным выделим __два итога__:
* Для разметки Золотого Стандарта возьмём набор UD. Почему? 1) Достаточно распространённый набор тегов. 2) Охватывает достаточно релевантных для русского языка частей речи (хотя вводные слова?). 3) Используется 2/3 библиотек, которые мы сравниваем. 4) Достаточно объёмный, использует разделение на более мелкие классы слов (например, ADV vs. ADV, INTJ, PART). Хотя есть и некоторые неясности: не всегда UD понимает теги и части речи так же, как они понимаются, например, в школьной традиции (первый, второй - это порядковые числительные, тогда как UD разметка даёт тег ADJ - прилагательное)
* С токенизацией есть проблема. Золотой Стандарт был токенизирован с помощью nltk, pymorphy2 так же в нашем случае будет работать от стороннего токенизатора. Остальные библиотеки имеют свои токенизаторы. Возможно, какой-то из токенизаторов работает по другим правилам. Как быть? Оставить как есть или в каждый морфологический анализатор подавать ранее токенизированный текст? Остановимся на втором варианте.

## Разметка парсеров

Прогоняем наш текст через парсеры:

### ``udpipe``

In [ ]:
# качаем модель для русского
name = 'rus_udpipe_model'
!wget -O {rus_udpipe} https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe?raw=true

In [4]:
# или пользуем уже скачанную
rus_udpipe = 'russian-ud-2.0-170801.udpipe'

In [5]:
rus_model = Model.load(rus_udpipe)
rus_pipeline = Pipeline(rus_model, 'generic_tokenizer', '', '', '')

In [6]:
toks_tags_text = rus_pipeline.process(text)
print(toks_tags_text[:1000])


# newdoc
# newpar
# sent_id = 1
# text = Первое что хочется сказать, так это то, что это очень необычный парк США.
1	Первое	ПЕРВЫЙ	ADJ	ORD	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	3	amod	_	_
2	что	ЧТО	PRON	WP	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	3	nsubj	_	_
3	хочется	хочется	VERB	VBC	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Mid	0	root	_	_
4	сказать	сказать	VERB	VB	Aspect=Imp|VerbForm=Inf	3	xcomp	_	SpaceAfter=No
5	,	,	PUNCT	,	_	8	punct	_	_
6	так	ТАК	SCONJ	IN	_	8	mark	_	_
7	это	ЭТО	PRON	DT	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	8	nsubj	_	_
8	то	ТО	PRON	DT	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	3	advcl	_	SpaceAfter=No
9	,	,	PUNCT	,	_	13	punct	_	_
10	что	ЧТО	SCONJ	IN	_	13	mark	_	_
11	это	ЭТО	PRON	DT	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	13	nsubj	_	_
12	очень	ОЧЕНЬ	ADV	RB	_	13	advmod	_	_
13	необычный	НЕОБЫЧНЫЙ	ADJ	JJL	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	8	ccomp	_	_
14	парк	ПАРК	NOUN	NN	Animacy=Inan|Case=Acc|Gender=M

In [9]:
# распарсим CoNLL-U формат
final_parse = conllu.parse(toks_tags_text)
final_parse[:5]

[TokenList<Первое, что, хочется, сказать, ,, так, это, то, ,, что, это, очень, необычный, парк, США, .>,
 TokenList<В, этом, парке, располагаются, 2, самых, активных, вулкана, в, мире, -, Килауэа, и, Мауна, -, Лоа, .>,
 TokenList<В, этом, парке, всегда, все, по, -, новому, .>,
 TokenList<Из, -, за, вулканических, извержений, тут, постоянно, меняется, пейзаж, .>,
 TokenList<Наверное, ,, только, в, этом, месте, понимаешь, ,, что, такое, природа, ,, а, что, такое, человек, .>]

Мы видим, что токенизация в данной библиотеке происходит по другим правилам, поэтому будем использовать заранее токенизированный текст:

In [26]:
list_parsed_tokens = []
for token in list_tokens:
    tagged_token = rus_pipeline.process(token)
    token_info = conllu.parse(tagged_token)
    list_parsed_tokens.append(token_info)



In [40]:
dict_udpipe_tags = {}
count = 0
for token in list_parsed_tokens:
    token_dict = token[0][0]
    dict_udpipe_tags[count] = [token_dict['form'], token_dict['upos'], token_dict['feats']]
    count += 1

print(len(dict_udpipe_tags))

365


### ``stanza``

In [43]:
# загружаем модель для русского
stanza.download('ru')

/usr/local/lib/python3.8/dist-packages/cryptography/hazmat/backends/openssl/x509.py:15: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
2021-10-03 14:57:22 INFO: Downloading default packages for language: ru (Russian)...
2021-10-03 15:12:55 INFO: Finished downloading models and saved to /home/yanina/stanza_resources.


``stanza`` требует для токенизатор для теггинга

In [45]:
nlp = stanza.Pipeline(lang='ru', processors='tokenize,pos')  # mwt для русского нет

2021-10-03 15:14:18 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-10-03 15:14:18 INFO: Use device: cpu
2021-10-03 15:14:18 INFO: Loading: tokenize
2021-10-03 15:14:18 INFO: Loading: pos
2021-10-03 15:14:19 INFO: Done loading processors!


Посмотрим, как токенизируется полный текст:

In [50]:
doc = nlp(text)
list_tokens_stanza = [word.text for sent in doc.sentences for word in sent.words]

print(list_tokens_stanza)
print(len(list_tokens_stanza))

['Первое', 'что', 'хочется', 'сказать', ',', 'так', 'это', 'то', ',', 'что', 'это', 'очень', 'необычный', 'парк', 'США', '.', 'В', 'этом', 'парке', 'располагаются', '2', 'самых', 'активных', 'вулкана', 'в', 'мире', '-', 'Килауэа', 'и', 'Мауна-Лоа', '.', 'В', 'этом', 'парке', 'всегда', 'все', 'по-новому', '.', 'Из-за', 'вулканических', 'извержений', 'тут', 'постоянно', 'меняется', 'пейзаж', '.', 'Наверное', ',', 'только', 'в', 'этом', 'месте', 'понимаешь', ',', 'что', 'такое', 'природа', ',', 'а', 'что', 'такое', 'человек', '.', 'Несмотря', 'на', 'свою', 'вулканическую', 'смертоносность', ',', 'тут', 'проживает', 'много', 'очень', 'редких', 'птиц', ',', 'которые', 'обитают', 'на', 'уникальных', 'лесах', '.', 'Также', 'только', 'в', 'этом', 'месте', 'вы', 'сможете', 'увидеть', 'по-настоящему', 'огромные', 'папоротники', '.', 'Создали', 'этот', 'удивительный', 'парк', 'в', '1916', 'г.', 'на', 'острове', 'Гавайи', ',', 'на', 'самом', 'крупном', 'из', 'Гавайских', 'островов', '.', 'Самым', 

Нам повезло: ``stanza`` имеет ту же (или похожую) систему токенизации текста. Тогда попробуем достать теги:

In [51]:
dict_stanza_tags = {}
count = 0
for sent in doc.sentences:
    for word in sent.words:
        dict_stanza_tags[count] = [word.text, word.upos, {word.feats if word.feats else "_"}]
        count += 1

print(len(dict_stanza_tags))

365


### ``pymorphy2``

In [52]:
morph = MorphAnalyzer()

In [154]:
# будем использовать самый вероятный разбор
dict_pymorphy_tags = {}
count = 0
for token in list_tokens:
    parse = morph.parse(token)[0]
    word = parse.word
    pos_tag = parse.tag.POS
    feats = parse.tag
    lemma = parse.normal_form
    if pos_tag == None:
        for tag in ['LATN', 'PNCT', 'NUMB', 'ROMN', 'UNKN']:
            if tag in feats:
                dict_pymorphy_tags[count] = [word, tag, feats, lemma]
    else:
        for tag in ['NOUN', 'ADJF', 'ADJS',
        'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS',
        'GRND', 'NUMR', 'ADVB', 'NPRO',
        'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']:
            if tag in feats:
                dict_pymorphy_tags[count] = [word, tag, feats, lemma]
    count += 1

print(len(dict_pymorphy_tags))

365


Разметка ``pymorphy2`` не соответствует разметке, которую мы приняли по умолчанию. Напишем код, который мог бы поменять теги ``pymorphy2`` на UD. Сначала создадим словарь однозначных соответствий тегов.

In [155]:
dict_conf_pos = {
    'ADJ': ['ADJF', 'ADJS', 'COMP', 'PRTF'],
    'VERB': ['INFN', 'PRTS', 'GRND'],
    'ADV': ['ADVB', 'PRED'], # есть сложности?
    'PRON': ['NPRO'],  # есть сложности
    'ADP': ['PREP']
}
dict_conf_tags = {
    'NUM': ['NUMB'],  # есть сложности
    'PUNCT': ['PNCT'],
    'X': ['UNKN', 'ROMN', 'LATN'],
}

Словарь однозначных соответствий получился не таким большим. Рассмотрим каверзные моменты:
* В изначальной системе есть только тег существительного. На деле система отличает и именованные сущности. Таким образом, если в морфологическом разборе есть 'Name sing'|'Name plur' - это имя собственное.
* Проблема с числительными заключается в том, что по UD некоторые числительные вроде сотня, тысяча имеют в разных ситуациях разные теги. В частности, если перед таким числительным-существительным стоит реальное число, то оно также получает тег NUM, в остальных случаях - это существительное.
* Неясно, что разбирается как предикатив (PRED) данной системой. Из примера понятно, что это могут быть адвербиалы. Вообще класс предикативов может включать в себя и другие части речи: прилагательные (депиктивы вроде *пришёл счастливым*), существительные (*стало лень*), причастия (краткие вроде *был прикован*). Исходя из эмпирических данных (см.ниже), предположим, что система умеет разбирать только случаи с адвербиалами.
* Анализатор не различает два вида союзов. Придётся прибегнуть к словарному методу: если что-то анализируемое есть в некотором списке, то это сочинительный союз.
* Другая ситуация с подчинительными союзами. Система UD отличает местоимения от простых подчинительных. Непонятно также, будет ли система оценивать подчинительные союзы как NPRO. В этом случае придумать правило достаточно сложно. 
* PRCL - это частица. Таким тегом разбираются стандартные частицы в русском, но не тот разбор, который предлагает UD. С точки зрения Universal Dependencies такие вещи разбиваются на 2 составляющие: AUX - вспомогательный глагол или частица, определяющие время или наклонение, и собственно PART. В принципе, можно прописать, что "бы", например, будет относиться к AUX.
* Кроме того, проблема существует и с глаголами в связи с делением на VERB и AUX. Глагол *быть* может выступать показателем будущего времени. Здесь можно придумать правило про n-граммы, состоящие из двух глаголов, среди которых будет глагол *быть*.

In [61]:
list_str = ['мне было лень это делать',
'мне стало плохо',
'он пришёл счастливым ко мне',
'мне было некогда ходить по магазинам']
list_parsed_sents = []

for str in list_str:
    list_parsed_tokens = []
    tokens = word_tokenize(str)
    for token in tokens:
        list_parsed_tokens.append(morph.parse(token))
    list_parsed_sents.append(list_parsed_tokens)

print([sent[2] for sent in list_parsed_sents])

[[Parse(word='лень', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='лень', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'лень', 13, 0),)), Parse(word='лень', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='лень', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'лень', 13, 3),)), Parse(word='лень', tag=OpencorporaTag('PRED,pres'), normal_form='лень', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'лень', 242, 0),)), Parse(word='лёнь', tag=OpencorporaTag('NOUN,anim,masc,Name sing,voct,Infr'), normal_form='лёня', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'лёнь', 407, 7),)), Parse(word='лёнь', tag=OpencorporaTag('NOUN,anim,masc,Name plur,gent'), normal_form='лёня', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'лёнь', 407, 9),)), Parse(word='лёнь', tag=OpencorporaTag('NOUN,anim,masc,Name plur,accs'), normal_form='лёня', score=0.16666666666666666, methods_stack=((<Dict

Несмотря на то, что таких каверзных моментов много, в некоторых на самом деле неясно, проблема ли это разметки или действительно системы. Иногда кажется, что такими правилами мы фактически решаем омонимию за систему. В данной работе мы пропишем эти правила, проверяя также, в принципе распознала ли система камень преткновения.

In [160]:
list_conf_pymorphy_tags = [tag for value in dict_conf_pos.values() for tag in value ]
print(list_conf_pymorphy_tags)
list_conf_unusual_tags = [tag for value in dict_conf_tags.values() for tag in value]
print(list_conf_unusual_tags)
quantifiers = ['тысяча', 'миллион', 'миллиард', 'триллион']  # взяты со страницы UD
list_values = [value for value in dict_pymorphy_tags.values()]
# print(list_values)

['ADJF', 'ADJS', 'COMP', 'PRTF', 'INFN', 'PRTS', 'GRND', 'ADVB', 'PRED', 'NPRO', 'PREP']
['NUMB', 'PNCT', 'UNKN', 'ROMN', 'LATN']


In [173]:
new_dict_pymorphy_tags = {}
count = 0

for ls in list_values:  # первый этап - меняем тривиальные вещи
    if ls[1] in list_conf_pymorphy_tags or ls[1] in list_conf_unusual_tags:
        for key, value in dict_conf.items():
            if ls[1] in value:
                ls[1] = key
        # print('Выход', ls[1])
        new_dict_pymorphy_tags[count] = ls
    else:
        new_dict_pymorphy_tags[count] = ls
    count += 1

# print(new_dict_pymorphy_tags)

full_dict_pymorphy_tags = {}

new_count = 0
new_values = [value for value in new_dict_pymorphy_tags.values()]
for ls in new_values:  # меняем нетривиальные вещи
    if ls[1] == 'NOUN':  # именованные сущности
        if 'Name' in ls[2] or 'Geox' in ls[2]:
            ls[1] = 'PROPN'
    if ls[1] == 'NUMR' and ls[3] in quantifiers:  # существительное или числительное
        idx_list = new_count
        idx_last_list = idx_list - 1
        if new_values[idx_last_list][1] != 'NUM':
            ls[1] = 'NOUN'
    if ls[1] == 'CONJ':
        if ls[3] in ['и', 'да', 'ни-ни', 'тоже', 'также',
        'а', 'но', 'зато', 'однако', 'же',
        'или', 'либо', 'то-то']:
            ls[1] = 'CCONJ'
        else:
            ls[1] = 'SCONJ'
    if ls[1] == 'PRCL':
        if ls[3] == 'бы':
            ls[1] = 'AUX'
        else:
            ls[1] = 'PART'
    if ls[1] == 'VERB' and ls[3] == 'быть' and 'futr' in ls[2]:
        idx_list = new_count
        idx_next_list = idx_list + 1
        if new_values[idx_last_list][1] == 'VERB':
            ls[1] = 'AUX'
    full_dict_pymorphy_tags[new_count] = ls

    new_count += 1

print(len(full_dict_pymorphy_tags))

365


## Сравнение и оценка ответов систем

In [131]:
with open('corpora.csv', encoding='utf-8') as corpora_csv:
    text = corpora_csv.read()

text = text.replace(',",', ',";')

In [132]:
text = text.replace('","', 'comma')
text = text.replace(',', ';')
text = text.replace('comma', ',')

In [134]:
dict_standard_tag = {}
splited_text = text.split('\n')
for line in splited_text:
    if splited_text.index(line) != 0:
        ls_line = line.split(';')
        dict_standard_tag[ls_line[0]] = ls_line[1]

In [183]:
def get_accuracy(system, standard):
    """
    Функция, подсчитывающая accuracy для систем
    """

    dict_2_sys = {}

    count = 0
    for my_tag in standard.values():
        sys_tag = system[count][1]
        dict_2_sys[count] = [my_tag, sys_tag]
        count += 1

    count_true = 0
    for value_pair in dict_2_sys.values():
        # print(value_pair)
        if value_pair[0] == value_pair[1]:
            count_true += 1

    accuracy = count_true / len(dict_2_sys)

    return accuracy

In [184]:
print('pymorphy', get_accuracy(full_dict_pymorphy_tags, dict_standard_tag))
print('udpipe', get_accuracy(dict_udpipe_tags, dict_standard_tag))
print('stanza', get_accuracy(dict_stanza_tags, dict_standard_tag))

pymorphy 0.1589958158995816
udpipe 0.14644351464435146
stanza 0.16736401673640167


Лучше всего по нашим расчётам справилась библиотека ``stanza``

## О предыдущем домашнем задании

Я могла бы предложить такие сочетания:
* PART + VERB (*не люблю*)
* PART + NOUN (*без надобности*, *к лицу*)
* PRON + ADV (*всё отлично*)